In [1]:
!pip install vllm datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 436.4/436.4 MB 255.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 449.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 315.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 888.1/888.1 MB 214.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 246.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 312.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 MB 272.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 227.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 263.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 304.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 287.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 307.1 MB/s eta 0:00

In [3]:
!pip install konlpy
!pip install mecab-python3


[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip


## 1. 테스트 데이터 전처리

In [4]:
import re
import json
import pandas as pd
from typing import List, Dict
from datasets import load_dataset, Dataset
from vllm import LLM, SamplingParams

INFO 09-23 01:51:14 [__init__.py:216] Automatically detected platform cuda.


In [5]:
def load_train_dataset_from_json(path: str):
    with open(path, "r", encoding="utf-8-sig") as f:
        data = json.load(f)
        return data

# 사용 예시
dataset = load_train_dataset_from_json("qwen3_company_train_dataset_combined.json")



In [6]:
dataset = Dataset.from_list(dataset)

In [7]:
print(len(dataset))

286


In [8]:
# 테스트 비율 설정
test_ratio = 1

# 전체 길이와 테스트 데이터 크기 계산
total_len = len(dataset)
test_size = int(total_len * test_ratio)

# 앞에서부터 테스트 데이터, 나머지는 학습 데이터
test_indices = list(range(test_size))
train_indices = list(range(test_size, total_len))

In [9]:
dataset[1]['messages'][1:]

[{'content': '개발 프로세스의 Git 워크플로우에서 `feature/{기능명}` 브랜치 생성 시 주의해야 할 점은 무엇인가요?',
  'role': 'user'},
 {'content': '<tool_call>{"name": "frontend_search", "arguments": {"keyword": "개발 프로세스 Git 워크플로우에서 feature 브랜치 생성 시 주의사항"}}</tool_call>',
  'role': 'assistant'},
 {'content': '<tool_response>검색 결과:\n-----\n### 3.3. 성능 최적화\n- **Lazy Loading**: 필요할 때만 컴포넌트를 로드하여 초기 로딩 속도 개선\n- **Memoization**: React의 `useMemo` 또는 `React.memo`를 사용하여 불필요한 리렌더링 방지\n\n## 4. 개발 프로세스\n\n### 4.1. Git 워크플로우\n- **브랜치 전략**: 기능 개발 시 `feature/{기능명}` 브랜치 생성\n- **Pull Request**: 코드 리뷰 후 `main` 브랜치에 병합\n\n### 4.2. 코드 리뷰\n- **리뷰 기준**: 코드 가독성, 성능, 보안 및 버그 여부 확인\n- **피드백 반영**: 리뷰어의 피드백을 수용하여 코드 수정\n\n### 4.3. 테스트\n- **유닛 테스트**: Jest 또는 Mocha를 사용하여 컴포넌트 단위 테스트 작성\n- **E2E 테스트**: Cypress 또는 Selenium을 사용하여 사용자 흐름 테스트\n\n## 5. 배포 및 운영 [[ref1]]\n<!-- 회사: 코드노바 | 대상: 사원(프론트엔드) | 작성일: 2025-08-29 -->\n# 빌드/배포 가이드\n분류: frontend | 회사: 코드노바 | 버전: v1.0 | 작성일: 2025-08-29\n\n---\n\n## 1. 빌드 준비\n\n### 1.1 환경 설정\n- **Node.js 설치**: 최신 LTS 버전을 

In [10]:
# OpenAI 포맷으로 변환 함수
def format_conversations(sample):
    return {
        "messages": [
            {"role": "system", "content": sample['messages'][0]['content']},
            *sample["messages"][1:]
        ]
    }

# 분할 및 변환
train_dataset = [format_conversations(dataset[i]) for i in train_indices]
test_dataset = [format_conversations(dataset[i]) for i in test_indices]

# 리스트를 다시 HuggingFace Dataset 객체로 변환
train_dataset = Dataset.from_list(train_dataset)
test_dataset = Dataset.from_list(test_dataset)

# 결과 확인
print(f"\n전체 데이터 분할 결과: Train {len(train_dataset)}개, Test {len(test_dataset)}개")


전체 데이터 분할 결과: Train 0개, Test 286개


In [11]:
def to_chatml(data):
    """
    data: messages 리스트이거나 {"messages": [...]} 형태의 dict
    반환값: ChatML 포맷의 문자열
    """
    # data가 dict이고 'messages' 키가 있으면 messages 리스트를 꺼내고,
    # 아니면 data 자체를 messages 리스트로 간주
    messages = data.get("messages") if isinstance(data, dict) and "messages" in data else data

    parts = []
    for msg in messages:
        role = msg["role"]
        content = msg["content"]
        parts.append(f"<|im_start|>{role}\n{content}<|im_end|>")
    return "\n".join(parts)

In [12]:
def extract_examples(chatml: str) -> List[Dict[str, str]]:
    """
    ChatML 문자열에서 각 assistant 응답을 분리하여
    'input'과 'label' 쌍을 생성합니다.
    'input'은 해당 assistant 응답 직전까지의 모든 대화 + '<|im_start|>assistant',
    'label'은 해당 assistant의 응답 내용입니다.
    """
    examples: List[Dict[str, str]] = []
    pattern = re.compile(r'<\|im_start\|>assistant(.*?)(?=<\|im_end\|>)', re.DOTALL)

    for match in pattern.finditer(chatml):
        start_idx = match.start()
        input_text = chatml[:start_idx].strip() + '\n<|im_start|>assistant'
        label_text = match.group(1).strip()
        examples.append({
            "input": input_text,
            "label": label_text
        })

    return examples

In [13]:
prompt_lst = []
label_lst = []

for item in test_dataset:
    chatml = to_chatml(item)  # ChatML 문자열로 변환
    examples = extract_examples(chatml)  # assistant 응답 단위로 분리

    for ex in examples:
        prompt_lst.append(ex['input'])
        label_lst.append(ex['label'])

In [14]:
print(prompt_lst[10])

<|im_start|>system

당신은 사내 지식을 활용하여 사용자의 질문에 정확하고 유용한 답변을 제공하는 코드노바의 사내 문서 AI 챗봇입니다.


사용자는 frontend(프론트엔드)팀에 속한 팀원입니다.
당신은 <tools></tools> 안에 있는 tool을 호출하여 문서를 검색할 수 있습니다.
일상적인 질문(ex: 안녕, 안녕하세요, 반가워 등)의 경우, tool 호출 없이 바로 답변하세요.

# Tools

You may call one or more functions to assist with the user query.

You are provided with function signatures within <tools></tools> XML tags:
<tools>
{"type": "function", "function": {"name": "frontend_search", "description": "사내 문서 검색을 위한 도구입니다. 대화 내역을 바탕으로 사용자가 원하는 문서를 찾고, 관련된 문서를 반환합니다.", "parameters": {"type": "object", "properties": {"keyword": {"type": "string", "description": "검색할 문서 키워드 (예: '코드노바 API 서버 설정')"}}, "required": ["keyword"], "additionalProperties": false}}}
</tools>

For each function call, return a json object with function name and arguments within <tool_call></tool_call> XML tags:
<tool_call>
{"name": <function-name>, "arguments": <args-json-object>}'
</tool_call>


그리고 다음 지침을 반드시 따르세요:
1. 기존의 말투는 잊고 정중하고 사무적인 어조로 답변해야 하세요.
2. 대

In [15]:
print(label_lst[8])

<tool_call>{"name": "frontend_search", "arguments": {"keyword": "릴리즈 노트 작성 체크리스트"}}</tool_call>


## 2. 모델 호출

In [16]:
sampling_params = SamplingParams(
    temperature=0,
    max_tokens=2048,
    stop=["<|im_end|>"]
)

In [17]:
llm = LLM(model="SKN14-Final-1Team/qwen3-8b-informal-formal-merged-09-19")

INFO 09-23 01:51:28 [utils.py:328] non-default args: {'disable_log_stats': True, 'model': 'SKN14-Final-1Team/qwen3-8b-informal-formal-merged-09-19'}


config.json: 0.00B [00:00, ?B/s]

INFO 09-23 01:51:42 [__init__.py:742] Resolved architecture: Qwen3ForCausalLM


`torch_dtype` is deprecated! Use `dtype` instead!


INFO 09-23 01:51:42 [__init__.py:1815] Using max model len 40960
INFO 09-23 01:51:44 [scheduler.py:222] Chunked prefill is enabled with max_num_batched_tokens=16384.


...

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


(EngineCore_DP0 pid=1551) INFO 09-23 01:53:55 [default_loader.py:268] Loading weights took 1.50 seconds
(EngineCore_DP0 pid=1551) INFO 09-23 01:53:56 [gpu_model_runner.py:2392] Model loading took 15.2683 GiB and 114.341844 seconds
(EngineCore_DP0 pid=1551) INFO 09-23 01:54:00 [backends.py:539] Using cache directory: /root/.cache/vllm/torch_compile_cache/d625b601a4/rank_0_0/backbone for vLLM's torch.compile
(EngineCore_DP0 pid=1551) INFO 09-23 01:54:00 [backends.py:550] Dynamo bytecode transform time: 3.50 s
(EngineCore_DP0 pid=1551) INFO 09-23 01:54:02 [backends.py:194] Cache the graph for dynamic shape for later use
(EngineCore_DP0 pid=1551) INFO 09-23 01:54:15 [backends.py:215] Compiling a graph for dynamic shape takes 15.03 s
(EngineCore_DP0 pid=1551) INFO 09-23 01:54:16 [monitor.py:34] torch.compile takes 18.52 s in total
(EngineCore_DP0 pid=1551) INFO 09-23 01:54:17 [gpu_worker.py:298] Available KV cache memory: 139.52 GiB
(EngineCore_DP0 pid=1551) INFO 09-23 01:54:18 [kv_cache_ut

Capturing CUDA graphs (mixed prefill-decode, PIECEWISE): 100%|██████████| 67/67 [00:01<00:00, 47.30it/s]


(EngineCore_DP0 pid=1551) INFO 09-23 01:54:20 [gpu_model_runner.py:3118] Graph capturing finished in 2 secs, took 0.72 GiB
(EngineCore_DP0 pid=1551) INFO 09-23 01:54:20 [gpu_worker.py:391] Free memory on device (177.65/178.35 GiB) on startup. Desired GPU memory utilization is (0.9, 160.52 GiB). Actual usage is 15.27 GiB for weight, 5.66 GiB for peak activation, 0.07 GiB for non-torch memory, and 0.72 GiB for CUDAGraph memory. Replace gpu_memory_utilization config with `--kv-cache-memory=148879492608` to fit into requested memory, or `--kv-cache-memory=167277282816` to fully utilize gpu memory. Current kv cache memory in use is 149812725248 bytes.
(EngineCore_DP0 pid=1551) INFO 09-23 01:54:20 [core.py:218] init engine (profile, create kv cache, warmup model) took 24.37 seconds
INFO 09-23 01:54:21 [llm.py:295] Supported_tasks: ['generate']
INFO 09-23 01:54:21 [__init__.py:36] No IOProcessor plugins requested by the model


In [18]:
fine_tuned_outputs = llm.generate(prompt_lst, sampling_params)

Adding requests:   0%|          | 0/824 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/824 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

In [19]:
fine_tuned_text_results = [sample.outputs[0].text.strip() for sample in fine_tuned_outputs]

In [20]:
print(fine_tuned_text_results[10])

<tool_call>{"name": "frontend_search", "arguments": {"keyword": "코드 리뷰 시 성능 평가 방법"}}</tool_call>
<tool_call>{"name": "frontend_search", "arguments": {"keyword": "코드 리뷰 시 보안 기준 평가 방법"}}</tool_call>
<tool_call>{"name": "frontend_search", "arguments": {"keyword": "코드 리뷰 체크리스트"}}</tool_call>


## 3. 평가 결과 저장

In [24]:
import pandas as pd
import re

# 예시 DataFrame
df = pd.DataFrame({
    "prompt": prompt_lst,
    "label": label_lst,
    "output": fine_tuned_text_results
})

# 'output' 컬럼에서 <think>...</think> 태그 제거
df['output'] = df['output'].apply(lambda x: re.sub(r"<think>.*?</think>", "", x, flags=re.S))

# 저장
df.to_csv("evaluation_results.csv", index=False, encoding="utf-8-sig")

In [25]:
df

,prompt,label,output
0,<|im_start|>system\n\n당신은 사내 지식을 활용하여 사용자의 질문에...,"<tool_call>{""name"": ""frontend_search"", ""argume...","<tool_call>{""name"": ""frontend_search"", ""argume..."
1,<|im_start|>system\n\n당신은 사내 지식을 활용하여 사용자의 질문에...,"프론트엔드 아키텍처 문서에 따르면, 성능 최적화를 위한 Lazy Loading은 필...","\n\n프론트엔드 아키텍처 문서에 따르면, 성능 최적화를 위한 Lazy Loadin..."
2,<|im_start|>system\n\n당신은 사내 지식을 활용하여 사용자의 질문에...,"<tool_call>{""name"": ""frontend_search"", ""argume...","<tool_call>{""name"": ""frontend_search"", ""argume..."
3,<|im_start|>system\n\n당신은 사내 지식을 활용하여 사용자의 질문에...,프론트엔드팀의 코드 리뷰 기준은 다음과 같습니다.\n\n- 모든 코드는 코드 리뷰를...,\n\n프론트엔드팀 코드 리뷰 기준은 다음과 같습니다.\n\n- 모든 코드는 코드 ...
4,<|im_start|>system\n\n당신은 사내 지식을 활용하여 사용자의 질문에...,"<tool_call>{""name"": ""frontend_search"", ""argume...","<tool_call>{""name"": ""frontend_search"", ""argume..."
...,...,...,...
819,<|im_start|>system\n\n당신은 사내 지식을 활용하여 사용자의 질문에...,주말에 방문할 만한 장소 추천에 관해서는 사내 문서에 관련 정보가 없어서 개인적인 ...,주말에 방문할 만한 장소 추천에 관해서는 사내 문서에 관련 정보가 없어서 구체적인 ...
820,<|im_start|>system\n\n당신은 사내 지식을 활용하여 사용자의 질문에...,"미안하지만, 나는 사내 문서 기반으로 정보를 주는 AI 챗봇이라 외부 최신 카페나 ...","미안하지만, 난 사내 문서 기반으로 업무 관련 정보만 알려주는 AI 챗봇이야. 카페..."
821,<|im_start|>system\n\n당신은 사내 지식을 활용하여 사용자의 질문에...,오늘 운동할 때 도움이 되는 기본 스트레칭 동작으로 '전신 스트레칭' 추천할게\n\...,\n\n운동할 때 도움 되는 간단한 스트레칭 동작으로 '목 돌리기' 추천할게.\n\...
822,<|im_start|>system\n\n당신은 사내 지식을 활용하여 사용자의 질문에...,"<tool_call>{""name"": ""cto_search"", ""arguments"":...","<tool_call>{""name"": ""cto_search"", ""arguments"":..."


In [21]:
for label, pred in zip(df['label'].to_list()[:50], df['output'].to_list()[:50]):
    print('레이블 :', label)
    print('--' * 50)
    print('모델의 예측:', pred)
    print('==' * 50)

레이블 : <tool_call>{"name": "frontend_search", "arguments": {"keyword": "빌드/배포 가이드의 1.1 환경 설정에서 Node.js 설치에 대한 구체적인 버전 요구사항"}}</tool_call>
----------------------------------------------------------------------------------------------------
모델의 예측: <tool_call>{"name": "frontend_search", "arguments": {"keyword": "빌드/배포 가이드의 1.1 환경 설정에서 Node.js 설치에 대한 구체적인 버전 요구사항"}}</tool_call>
레이블 : 빌드/배포 가이드의 '1.1 환경 설정' 파트에 따르면, Node.js는 최신 LTS(Long Term Support) 버전을 설치해야 합니다. 구체적인 버전 번호보다는 최신 LTS 버전을 사용하는 것이 요구사항입니다.
----------------------------------------------------------------------------------------------------
모델의 예측: 

빌드/배포 가이드의 '1.1 환경 설정' 파트에 따르면, Node.js는 최신 LTS(LTS) 버전을 설치하는 것이 요구사항입니다. 구체적인 버전 번호는 명시되어 있지 않으나, 최신 LTS 버전을 권장하고 있습니다.
레이블 : 오늘 저녁 메뉴로는 건강과 균형을 고려하여 단백질과 채소가 풍부한 그릴드 치킨 샐러드나 연어 스테이크를 추천드립니다. 간단하면서도 영양가 있는 식사를 원하신다면 이 메뉴가 적합할 것입니다.
----------------------------------------------------------------------------------------------------
모델의 예측: 

오늘 저녁 메뉴로는 균형 잡힌 영양과 맛을 고려하여 닭가슴살 샐러드나 

## 4. 평가

아래 코드는 펑션 콜링 성능을 평가하기 위한 Python 함수를 구현한 것입니다. 요청하신 세 가지 메트릭을 다음과 같이 구현했습니다:

- tool_selection: 함수 이름의 일치 여부를 평가합니다.
- params_selection: 파라미터 키(예: user_id)의 일치 여부를 평가합니다.
- params_value_accuracy: 파라미터 값(예: "U002")의 일치 여부를 평가합니다.

In [28]:
import re
import json
import unicodedata
from difflib import SequenceMatcher

# ======================
# 선택적 형태소 분석기 준비 (Okt -> Mecab -> fallback)
# ======================
def _try_import_tokenizers():
    okt = None
    mecab = None
    try:
        from konlpy.tag import Okt
        okt = Okt()
    except Exception:
        pass
    if okt is None:
        try:
            from konlpy.tag import Mecab
            mecab = Mecab()
        except Exception:
            pass
    return okt, mecab

_OKT, _MECAB = _try_import_tokenizers()
_PUNCT_RE = re.compile(r"[^\w\s]", flags=re.UNICODE)
_TOOL_CALL_RE = re.compile(r"<tool_call>(.*?)</tool_call>", re.DOTALL)
_THINK_TAG_RE = re.compile(r"<think>.*?</think>", re.S)

# ======================
# 전처리 & 토크나이즈 & 유사도
# ======================
def normalize_text(s):
    if s is None:
        return ""
    if not isinstance(s, str):
        try:
            s = json.dumps(s, ensure_ascii=False, sort_keys=True)
        except Exception:
            s = str(s)
    s = s.strip()
    s = unicodedata.normalize("NFC", s)
    return s

def tokenize_ko(s: str):
    s = normalize_text(s)
    s_clean = _PUNCT_RE.sub(" ", s)
    if _OKT is not None:
        try:
            return _OKT.morphs(s_clean)
        except Exception:
            pass
    if _MECAB is not None:
        try:
            return _MECAB.morphs(s_clean)
        except Exception:
            pass
    return s_clean.split()

def jaccard(a_tokens, b_tokens):
    a, b = set(a_tokens), set(b_tokens)
    if not a and not b:
        return 1.0
    if not a or not b:
        return 0.0
    inter = len(a & b)
    union = len(a | b)
    return inter / union if union else 0.0

def char_ratio(a: str, b: str):
    a = normalize_text(a)
    b = normalize_text(b)
    return SequenceMatcher(None, a, b).ratio()

def morph_similarity(a, b, alpha=0.6):
    """
    형태소 Jaccard(alpha) + 문자 유사도(1-alpha)
    """
    a_tokens = tokenize_ko(a)
    b_tokens = tokenize_ko(b)
    j = jaccard(a_tokens, b_tokens)
    c = char_ratio(a, b)
    return alpha * j + (1 - alpha) * c

# ======================
# 보조: <think> 제거
# ======================
def strip_think(s: str) -> str:
    if not isinstance(s, str):
        return s
    return _THINK_TAG_RE.sub("", s)

# ======================
# 메인 평가 함수 (유사도 점수만)
# ======================
def evaluate_function_calls_similarity(labels, predictions, alpha=0.6):
    results = {
        'tool_selection': {'correct': 0, 'total': 0},
        'params_selection': {'correct': 0, 'total': 0},
        'params_value_similarity': {'sum': 0.0, 'count': 0},
    }
    tool_call_count = 0

    for label_raw, pred_raw in zip(labels, predictions):
        label = strip_think(label_raw or "")
        pred  = strip_think(pred_raw or "")

        label_m = _TOOL_CALL_RE.search(label)
        pred_m  = _TOOL_CALL_RE.search(pred)

        if not label_m:
            continue
        tool_call_count += 1

        if not pred_m:
            results['tool_selection']['total'] += 1
            results['params_selection']['total'] += 1
            continue

        try:
            label_json = json.loads(label_m.group(1))
            pred_json  = json.loads(pred_m.group(1))
        except json.JSONDecodeError:
            results['tool_selection']['total'] += 1
            results['params_selection']['total'] += 1
            continue

        # 1) 함수 이름 일치 정확도
        results['tool_selection']['total'] += 1
        if label_json.get('name') == pred_json.get('name'):
            results['tool_selection']['correct'] += 1

        # 2) 파라미터 키 일치 정확도
        label_args = label_json.get('arguments', {}) or {}
        pred_args  = pred_json.get('arguments', {}) or {}
        label_keys = set(label_args.keys())
        pred_keys  = set(pred_args.keys())

        for k in label_keys:
            results['params_selection']['total'] += 1
            if k in pred_keys:
                results['params_selection']['correct'] += 1
        for k in pred_keys:
            if k not in label_keys:
                results['params_selection']['total'] += 1

        # 3) 값 유사도 (공통 키만)
        common = label_keys & pred_keys
        if common:
            sims = []
            for k in common:
                sims.append(morph_similarity(label_args.get(k), pred_args.get(k), alpha=alpha))
            sample_sim = sum(sims) / len(sims)
            results['params_value_similarity']['sum'] += sample_sim
            results['params_value_similarity']['count'] += 1

    final = {}
    for metric in ('tool_selection', 'params_selection'):
        c = results[metric]['correct']
        t = results[metric]['total']
        final[metric] = (c / t) if t else 0.0

    sim_sum = results['params_value_similarity']['sum']
    sim_cnt = results['params_value_similarity']['count']
    final['params_value_similarity'] = (sim_sum / sim_cnt) if sim_cnt else 0.0

    final['total_samples'] = tool_call_count
    return final


In [29]:
labels = df['label'].to_list()
preds = df['output'].to_list()

results_with_errors = evaluate_function_calls_similarity(labels, preds, alpha=0.6)
for metric, value in results_with_errors.items():
    if metric != 'total_samples':
        print(f"{metric}: {value:.2%}")
    else:
        print(f"{metric}: {value}")

tool_selection: 98.05%
params_selection: 98.05%
params_value_similarity: 88.50%
total_samples: 308
